In [1]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model=deployment_name,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
        )

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Define a tool that gets the weather for a city.
async def get_weather(city: str) -> str:
    """Get the weather for a city."""
    return f"The weather in {city} is 72 degrees and Sunny."


# Create an assistant agent.
weather_agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    tools=[get_weather],
    system_message="Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition.
text_termination = TextMentionTermination("TERMINATE")

# Create a single-agent team.
single_agent_team = RoundRobinGroupChat([weather_agent], termination_condition=text_termination)

async def run_team() -> None:
    result = await single_agent_team.run(task="What is the weather in New York?")
    print(result)


# Use `asyncio.run(run_team())` when running in a script.
await run_team()


messages=[TextMessage(id='41da850d-91fa-4fd5-b662-7ca503f906c5', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 23, 0, 24, 41, 1125, tzinfo=datetime.timezone.utc), content='What is the weather in New York?', type='TextMessage'), ToolCallRequestEvent(id='45af90f4-917a-4e51-bc56-e663f5245afc', source='assistant', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=16), metadata={}, created_at=datetime.datetime(2025, 7, 23, 0, 24, 43, 80618, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_4jFqlysx2i0WIgmUgoZbL2nb', arguments='{"city":"New York"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='4ea5a907-1a47-445b-bfc9-31bec974af72', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 23, 0, 24, 43, 81958, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', name='get_weather', call_id='call

In [3]:
from autogen_agentchat.base import TaskResult


async def run_team_stream() -> None:
    async for message in single_agent_team.run_stream(task="What is the weather in New York?"):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)


# Use `asyncio.run(run_team_stream())` when running in a script.
await run_team_stream()



id='b52213e3-6303-4e18-ab37-79ee3ba6dba6' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 15, 5, 41, 36, 138224, tzinfo=datetime.timezone.utc) content='What is the weather in New York?' type='TextMessage'
id='d6ad608d-1138-48f0-b8ae-73e5d8d0d0fc' source='assistant' models_usage=RequestUsage(prompt_tokens=594, completion_tokens=16) metadata={} created_at=datetime.datetime(2025, 7, 15, 5, 41, 36, 843192, tzinfo=datetime.timezone.utc) content=[FunctionCall(id='call_inCIeguzk4FyMbhmudyqo2Qt', arguments='{"city":"New York"}', name='get_weather')] type='ToolCallRequestEvent'
id='250782db-1ef7-4547-8d0c-f28ff93ba5f4' source='assistant' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 15, 5, 41, 36, 845504, tzinfo=datetime.timezone.utc) content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', name='get_weather', call_id='call_inCIeguzk4FyMbhmudyqo2Qt', is_error=False)] type='ToolCallExecutionEvent'
id='434

In [4]:
from autogen_agentchat.ui import Console

# Use `asyncio.run(single_agent_team.reset())` when running in a script.
await single_agent_team.reset()  # Reset the team for the next run.
# Use `asyncio.run(single_agent_team.run_stream(task="What is the weather in Seattle?"))` when running in a script.
await Console(
    single_agent_team.run_stream(task="What is the weather in Seattle?")
)  # Stream the messages to the console.

---------- TextMessage (user) ----------
What is the weather in Seattle?
---------- ToolCallRequestEvent (assistant) ----------
[FunctionCall(id='call_9rkrFTMoXlhVz3q0ph9AuSEb', arguments='{"city":"Seattle"}', name='get_weather')]
---------- ToolCallExecutionEvent (assistant) ----------
[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', name='get_weather', call_id='call_9rkrFTMoXlhVz3q0ph9AuSEb', is_error=False)]
---------- ToolCallSummaryMessage (assistant) ----------
The weather in Seattle is 72 degrees and Sunny.
---------- TextMessage (assistant) ----------
The weather in Seattle is currently 72 degrees and sunny.
---------- TextMessage (assistant) ----------
The current weather in Seattle is 72 degrees and sunny.
---------- TextMessage (assistant) ----------
The weather in Seattle is currently 72 degrees and sunny.
---------- TextMessage (assistant) ----------
The current weather in Seattle is 72 degrees and sunny.
---------- TextMessage (assistant

TaskResult(messages=[TextMessage(id='0a001986-9991-44a5-8c3b-137f6456d4c9', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 41, 48, 612123, tzinfo=datetime.timezone.utc), content='What is the weather in Seattle?', type='TextMessage'), ToolCallRequestEvent(id='9c05d252-2c1c-48e9-af9e-eb04848a9661', source='assistant', models_usage=RequestUsage(prompt_tokens=69, completion_tokens=15), metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 41, 49, 199258, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_9rkrFTMoXlhVz3q0ph9AuSEb', arguments='{"city":"Seattle"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='bcf3d72d-b737-4652-b298-d01717b6ff2b', source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 5, 41, 49, 200861, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', name='get_weather', c

In [7]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(15)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Create a team with the primary and critic agents.
reflection_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=termination)

In [8]:
# Use `asyncio.run(Console(reflection_team.run_stream(task="Write a short poem about fall season.")))` when running in a script.
await Console(
    reflection_team.run_stream(task="가을에 대한 짧은 시를 써보세요.")
)  # Stream the messages to the console.

---------- TextMessage (user) ----------
가을에 대한 짧은 시를 써보세요.
---------- TextMessage (primary) ----------
가을 바람 살랑이며  
낙엽은 춤추고  
황금빛 햇살 아래  
마음은 따스해지네.
---------- TextMessage (critic) ----------
아름다운 가을의 정취가 잘 담긴 시입니다. 짧지만 가을 바람의 부드러움과 낙엽의 춤, 햇살의 따스함이 생생하게 느껴집니다. 다만, 시의 이미지들이 조금 더 구체적이고 독창적으로 표현된다면 더욱 인상적일 것 같습니다. 예를 들어, 낙엽의 색이나 소리를 묘사하거나, 가을과 연결된 특별한 감정을 덧붙여 보세요. 전반적으로 매우 좋습니다!
---------- TextMessage (primary) ----------
감사합니다! 더 구체적이고 독창적인 표현을 살려서 가을 시를 다시 써보겠습니다.

---

바스락거리는 주홍빛 낙엽들  
차가운 바람에 속삭이며  
마음 한켠엔 서늘한 그리움  
가을은 조용히 스며드네.
---------- TextMessage (critic) ----------
훨씬 더 구체적이고 감성적인 표현이 돋보이는 시입니다. 낙엽의 색깔과 소리를 섬세하게 묘사했고, 바람과 마음의 감정이 자연스럽게 어우러져 가을의 깊은 분위기를 잘 전달합니다. ‘서늘한 그리움’이라는 표현이 특히 인상적이며, 시에 감정적인 깊이를 더해 주네요. 매우 좋은 발전입니다! 계속해서 창작 활동 응원합니다. APPROVE


TaskResult(messages=[TextMessage(id='0640517f-aa04-4e47-ab16-303714f22e4e', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 18, 2, 41, 59, 373141, tzinfo=datetime.timezone.utc), content='가을에 대한 짧은 시를 써보세요.', type='TextMessage'), TextMessage(id='f2ace8c5-80b9-4f46-95fd-bb7caf990dba', source='primary', models_usage=RequestUsage(prompt_tokens=33, completion_tokens=36), metadata={}, created_at=datetime.datetime(2025, 7, 18, 2, 42, 0, 357999, tzinfo=datetime.timezone.utc), content='가을 바람 살랑이며  \n낙엽은 춤추고  \n황금빛 햇살 아래  \n마음은 따스해지네.', type='TextMessage'), TextMessage(id='7daf26a3-cb80-4b7f-a1a6-467260691b32', source='critic', models_usage=RequestUsage(prompt_tokens=86, completion_tokens=121), metadata={}, created_at=datetime.datetime(2025, 7, 18, 2, 42, 2, 194920, tzinfo=datetime.timezone.utc), content='아름다운 가을의 정취가 잘 담긴 시입니다. 짧지만 가을 바람의 부드러움과 낙엽의 춤, 햇살의 따스함이 생생하게 느껴집니다. 다만, 시의 이미지들이 조금 더 구체적이고 독창적으로 표현된다면 더욱 인상적일 것 같습니다. 예를 들어, 낙엽의 색이나 소리를 묘사하거나, 가을과 연결된 특

In [10]:
await Console(reflection_team.run_stream(task="중국 당나라 시풍으로 시를 써보세요."))

---------- TextMessage (user) ----------
중국 당나라 시풍으로 시를 써보세요.
---------- TextMessage (primary) ----------
산間秋色深,  
한가로운 구름 흘러가네.  
고향 달빛 차가워도,  
외로운 마음은 평온하네.
---------- TextMessage (critic) ----------
시의 구성이 당나라 시풍의 느낌을 잘 살리고 있습니다. 산속의 깊은 가을 풍경과 한가로운 구름, 차가운 달빛이 어우러져 고요한 분위기를 자아내며, 외로운 마음이 평온함으로 변하는 섬세한 감정을 표현한 점이 인상적입니다. 다만, 당나라 시의 전통적인 특징 중 하나인 시적인 언어와 이미지의 운율감을 조금 더 살린다면 더욱 깊이 있는 작품이 될 것입니다. 예를 들어, 각 구절의 음절 수를 맞추거나, 병렬 구조를 강화하는 방식을 고려해보세요. 전반적으로 훌륭한 시이며, 계속해서 발전 가능성이 큽니다!
---------- TextMessage (primary) ----------
감사합니다! 지적해 주신 운율과 병렬 구조를 고려하여 당나라 시풍에 맞게 다시 써보았습니다.

---

산중秋色정深深(산중추색정심심),  
한가로이 구름 한 줄 흘러가네.  
달빛은 차가워도 고요하니,  
외로운 마음 바람에 잠기네.  

---

각 구절의 음절 수를 유사하게 맞추고, 병렬 구조를 강조하여 고전 당시의 느낌을 살리고자 했습니다. 더 원하시는 스타일이나 주제가 있다면 언제든 말씀해 주세요!
---------- TextMessage (critic) ----------
수정하신 시에서 운율과 병렬 구조가 더욱 정돈되어 당나라 시풍의 고전미가 잘 살아났습니다. "산중秋色정深深"과 같은 병렬적 표현과 "한가로이 구름 한 줄 흘러가네" 등의 부드러운 흐름이 자연 풍경과 마음의 고요함을 효과적으로 전달하고 있습니다. 또한 각 구절이 비슷한 음절 수를 지니면서 균형감이 느껴져 읽는 이로 하여금 편안함과 동시에 깊은 감상을 할 수 있게 

TaskResult(messages=[TextMessage(id='69c2709d-c747-4719-b4b6-f10caf223bfd', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 18, 2, 43, 12, 290990, tzinfo=datetime.timezone.utc), content='중국 당나라 시풍으로 시를 써보세요.', type='TextMessage'), TextMessage(id='3b10d338-c9dc-4c13-95f0-00a845df163f', source='primary', models_usage=RequestUsage(prompt_tokens=727, completion_tokens=39), metadata={}, created_at=datetime.datetime(2025, 7, 18, 2, 43, 13, 667182, tzinfo=datetime.timezone.utc), content='산間秋色深,  \n한가로운 구름 흘러가네.  \n고향 달빛 차가워도,  \n외로운 마음은 평온하네.', type='TextMessage'), TextMessage(id='be5a9518-8b7f-4be8-b516-90b9da2f10d2', source='critic', models_usage=RequestUsage(prompt_tokens=783, completion_tokens=165), metadata={}, created_at=datetime.datetime(2025, 7, 18, 2, 43, 15, 855648, tzinfo=datetime.timezone.utc), content='시의 구성이 당나라 시풍의 느낌을 잘 살리고 있습니다. 산속의 깊은 가을 풍경과 한가로운 구름, 차가운 달빛이 어우러져 고요한 분위기를 자아내며, 외로운 마음이 평온함으로 변하는 섬세한 감정을 표현한 점이 인상적입니다. 다만, 당나라 시의 전통적인 특징 중

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

# Create an OpenAI model client.
model_client = azure_openai_chat_completion_client

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    system_message="Always transfer to user when you don't know the answer. Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")
termination = handoff_termination | text_termination

# Create a single-agent team.
lazy_agent_team = RoundRobinGroupChat([lazy_agent], termination_condition=termination)

In [6]:
from autogen_agentchat.ui import Console

await Console(lazy_agent_team.run_stream(task="서울 날씨 어때?"),  output_stats=True)

# Handoff to the user before answering the question

await Console(lazy_agent_team.run_stream(task="날씨 맑은데?"))


---------- TextMessage (user) ----------
서울 날씨 어때?
---------- ToolCallRequestEvent (lazy_assistant) ----------
[FunctionCall(id='call_Ea8cU6wsvUJlx2w1qRYB8P99', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 174, Completion tokens: 12]
---------- ToolCallExecutionEvent (lazy_assistant) ----------
[FunctionExecutionResult(content='Transfer to user.', name='transfer_to_user', call_id='call_Ea8cU6wsvUJlx2w1qRYB8P99', is_error=False)]
---------- HandoffMessage (lazy_assistant) ----------
Transfer to user.
---------- Summary ----------
Number of messages: 4
Finish reason: Handoff to user from lazy_assistant detected.
Total prompt tokens: 174
Total completion tokens: 12
Duration: 0.56 seconds
---------- TextMessage (user) ----------
날씨 맑은데?
---------- TextMessage (lazy_assistant) ----------
현재 서울 날씨가 맑다고 하시니 다행이네요! 더 궁금한 점이나 다른 정보가 필요하시면 말씀해 주세요.
---------- TextMessage (lazy_assistant) ----------
현재 서울 날씨가 맑다고 하시니 다행이네요! 더 궁금한 점이나 다른 정보가 필요하시면 말씀해 주세요.
---------- TextMessage (lazy

TaskResult(messages=[TextMessage(id='c36c8791-485f-4790-86c6-629e5a7f9226', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 23, 0, 49, 18, 851633, tzinfo=datetime.timezone.utc), content='날씨 맑은데?', type='TextMessage'), TextMessage(id='5089598f-ee91-4807-b50e-34ddf50f862a', source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=210, completion_tokens=34), metadata={}, created_at=datetime.datetime(2025, 7, 23, 0, 49, 19, 484461, tzinfo=datetime.timezone.utc), content='현재 서울 날씨가 맑다고 하시니 다행이네요! 더 궁금한 점이나 다른 정보가 필요하시면 말씀해 주세요.', type='TextMessage'), TextMessage(id='fa9c9db0-5d33-4aa1-ad1c-72703475eada', source='lazy_assistant', models_usage=RequestUsage(prompt_tokens=246, completion_tokens=34), metadata={}, created_at=datetime.datetime(2025, 7, 23, 0, 49, 20, 138716, tzinfo=datetime.timezone.utc), content='현재 서울 날씨가 맑다고 하시니 다행이네요! 더 궁금한 점이나 다른 정보가 필요하시면 말씀해 주세요.', type='TextMessage'), TextMessage(id='a536c9ad-3b47-4071-bd0b-8e05f987cd00', source